In [54]:
import pandas as pd 
from sklearn.externals import joblib
import numpy as np


In [40]:
df_week = pd.read_csv("../data/16-17/week1.csv")

In [41]:
def get_teams(df):
    
    teams = []
    
    for i,cal in enumerate(df.iterrows()):
        teams.append(cal[1]["HomeTeam"])
        teams.append(cal[1]["AwayTeam"])

        if i == 9: 
            return teams


In [42]:
teams_1617 = get_teams(df_week)

print teams_1617

['Juventus', 'Fiorentina', 'Roma', 'Udinese', 'Atalanta', 'Lazio', 'Bologna', 'Crotone', 'Chievo', 'Inter', 'Empoli', 'Sampdoria', 'Genoa', 'Cagliari', 'Milan', 'Torino', 'Palermo', 'Sassuolo', 'Pescara', 'Napoli']


In [43]:
def build_features(df, teams):

    team_features = []
    
    for team in teams:
        #print team_a
        team_home = df[df['HomeTeam']==team]
        team_away = df[df['AwayTeam']==team]
        
        #shots made
        team_s    = team_away["AS"].sum()  + team_home["HS"].sum()
        #shots on-target made
        team_st   = team_away["AST"].sum() + team_home["HST"].sum()
        #shots conceded
        team_sc    = team_away["HS"].sum()  + team_home["AS"].sum()
        #shots on-target conceded
        team_stc   = team_away["HST"].sum() + team_home["AST"].sum()
        #corners awarded
        team_c    = team_away["AC"].sum()  + team_home["HC"].sum()
        #corners conceded
        team_cc    = team_away["HC"].sum()  + team_home["AC"].sum()

        team_features.append([team_s,team_sc,team_st,team_stc,team_c,team_cc])

    return team_features

In [44]:
def build_target(df, teams):

    team_target = []
    
    for team in teams:
        #print team_a
        t      = df[(df['HomeTeam']==team) | (df['AwayTeam']==team)]
        team_home = df[df['HomeTeam']==team]
        team_away = df[df['AwayTeam']==team]

        team_h_win = len(team_home[team_home['FTHG']>team_home['FTAG']])
        team_a_win = len(team_away[team_away['FTAG']>team_away['FTHG']])
        team_draw = len(t[t['FTAG']==t['FTHG']])

        team_points = 3*team_a_win + 3*team_h_win + team_draw
        team_target.append(team_points)

    return team_target


In [45]:
targ_1617 = build_target(df_week, teams_1617)
feat_1617 = build_features(df_week, teams_1617)

In [46]:
model_1 = joblib.load('../data/linreg_model.pkl')

pred_1617 = model_1.predict(feat_1617)

In [47]:
#print pred_1617
#print teams_1617
ranking = []

for t,p in zip(teams_1617,pred_1617):
    ranking.append((t, p)) 

In [48]:
ranking.sort(key=lambda x: x[1],reverse=True)
for t,p in ranking:
    print t,p

Roma 3.95198379888
Sassuolo 2.92885729694
Bologna 2.53529738093
Milan 2.4477985898
Genoa 1.67571738486
Lazio 1.62321197077
Sampdoria 1.5977900153
Napoli 1.57957979218
Chievo 1.51409865929
Pescara 1.50876414608
Juventus 1.42248187739
Atalanta 1.39430750977
Inter 1.14267098657
Torino 1.12767683926
Cagliari 0.852399158452
Empoli 0.682577833982
Fiorentina 0.60329818021
Palermo 0.11151318736
Crotone -0.122093480467
Udinese -0.249406825281


In [55]:
print targ_1617
#Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % model_1.score(feat_1617, targ_1617))
print("Residual sum of squares: %.2f"
      % np.mean((model_1.predict(feat_1617) - targ_1617) ** 2))

[3, 0, 3, 0, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 3, 0, 0, 3, 1, 1]
Variance score: 0.55
Residual sum of squares: 0.93


In [22]:
model_1 = joblib.load('../data/regtree_model.pkl')

pred_1617 = model_1.predict(feat_1617)

In [23]:
print pred_1617
print targ_1617

[ 43.35714286  43.35714286  43.35714286  43.35714286  43.35714286
  43.35714286  43.35714286  43.35714286  43.35714286  43.35714286
  43.35714286  43.35714286  43.35714286  43.35714286  43.35714286
  43.35714286  43.35714286  43.35714286  43.35714286  43.35714286]
[3, 0, 3, 0, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 3, 0, 0, 3, 1, 1]
